# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902074


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/41 [00:00<?, ?it/s]

Rendering models:   5%|▍         | 2/41 [00:04<01:20,  2.07s/it]

Rendering models:   7%|▋         | 3/41 [00:05<01:10,  1.86s/it]

Rendering models:  15%|█▍        | 6/41 [00:05<00:46,  1.32s/it]

Rendering models:  20%|█▉        | 8/41 [00:05<00:31,  1.05it/s]

...

Rendering models:  76%|███████▌  | 31/41 [00:09<00:01,  6.15it/s]

Rendering models:  80%|████████  | 33/41 [00:10<00:01,  7.54it/s]

Rendering models:  90%|█████████ | 37/41 [00:10<00:00,  9.28it/s]

Rendering models:  95%|█████████▌| 39/41 [00:10<00:00,  8.60it/s]

equidad1                              0.226882
welshgirlie                           0.165119
kayleebug2017                         0.004733
Lukes0404                             0.002919
not-logged-in-cae133427d99aac1e554    0.000865
Drew_Farnsworth                       0.005733
3lite                                 0.004514
Thedunkmasta                          0.004947
Thedunkmasta                          0.004706
Thedunkmasta                          0.004918
Thedunkmasta                          0.004327
Thedunkmasta                          0.005271
Thedunkmasta                          0.005235
Thedunkmasta                          0.004640
Thedunkmasta                          0.005000
Thedunkmasta                          0.004652
not-logged-in-7526601586eaa938b3ee    0.004705
Thedunkmasta                          0.004627
not-logged-in-c66f4fc0a6fdbc2ff731    0.006013
cmk24                                 0.000524
Jessica_Korkmaz                       0.005915
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())